In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("/home/florian/bachelorarbeit/code/Cross-Silo-FL/datasets/horizontal/covid/owid-covid-data.csv")

In [3]:
data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [4]:

import sys
sys.path.insert(1, '../scripts')
import helper


2023-01-02 18:19:02.942428: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-02 18:19:03.418237: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-02 18:19:03.418257: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-02 18:19:04.618497: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [5]:
x_train, y_train = helper.get_samples(data, 5, ["total_cases", "new_cases"], "new_cases", helper.Dataset.Covid, max_samples=1000)

In [6]:
x_train[10:20]

[array([5., 0., 5., 0., 8., 3., 8., 0.]),
 array([5., 0., 8., 3., 8., 0., 8., 0.]),
 array([8., 3., 8., 0., 8., 0., 8., 0.]),
 array([ 8.,  0.,  8.,  0.,  8.,  0., 11.,  3.]),
 array([ 8.,  0.,  8.,  0., 11.,  3., 11.,  0.]),
 array([ 8.,  0., 11.,  3., 11.,  0., 11.,  0.]),
 array([11.,  3., 11.,  0., 11.,  0., 14.,  3.]),
 array([11.,  0., 11.,  0., 14.,  3., 20.,  6.]),
 array([11.,  0., 14.,  3., 20.,  6., 25.,  5.]),
 array([14.,  3., 20.,  6., 25.,  5., 26.,  1.])]

In [7]:
y_train[0:5]

[0.0, 0.0, 0.0, 0.0, 0.0]

*Scikit-Learn tests start here*

In [58]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error

import tensorflow as tf

In [85]:
np.array(x_train).shape[1]

8